In [253]:
import pandas as pd
import os
from zipfile import ZipFile
from bs4 import BeautifulSoup

# Analysis on data dump
This notebook is to learn reading data dumps directly form the server (Rnd2) as opposed to obtain the data through API calls

Recent data dump with normalized lang tags produced at the end of June 2022 for the project Etranslate

In [254]:
#data dump location
data_path="/projects/etranslate-data-dump"

In [255]:
#listing files in directory
list_data_files=os.listdir(data_path)
list_data_files[2]

'00719.zip'

In [256]:
#locating zip file. Each zip file  contains a collection
file_name = os.path.join(data_path,  list_data_files[2])

In [ ]:
#reading files that

In [264]:
# extrecting  files that have not been extracted
with ZipFile(file_name, 'r') as zipObject:
 listOfFileNames = zipObject.namelist()
 for fileName in listOfFileNames:
        if os.path.isfile(fileName):
           # Extract a single file from zip
            print('file already exist')
        else:
            zipObject.extract(fileName)
            print('file extracted')

file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist


In [265]:
#open one of the xml unzipped files
with open(listOfFileNames[0], 'r') as f:
	file = f.read() 

In [266]:
#use beaut soup for parsing xml
soup = BeautifulSoup(file, 'xml')

In [267]:
#finding all description tag
t=soup.find('title')

In [268]:
t

<dc:title>Entwürfe des Hauptstaatsarchivs Stuttgart für das Landeswappen, insbesondere auf der Grundlage der Wappen von Baden, Württemberg und dem Herzogtum Schwaben, Frühe Versuche einer Kombination des badischen und des württembergischen Wappens von Hansmartin Decker-Hauff</dc:title>

In [269]:
# find attributes of title
t.attrs

{}

In [ ]:
#alternative to find keyword in file?
keyword = 'your_keyword'
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        f = open(os.path.join(your_path, file),'r')
        for x in f:
            if keyword in x:
                #do what you want
        f.close()